# Data Preprocessing

This file walks you through how I preprocessed the customer_data.csv file and export customer_data_preprocessed.csv for the subsequent exploratory data anlysis and RFM analysis.

In [91]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

In [67]:
# import dataset
df = pd.read_csv('customer_data.csv', delimiter='\t')
df.head()

,ID,signup_ym,birth_year,annual_income,marital_status,children,recency,amount_alcohol,amount_fruit,amount_meat,...,num_purchase_web,num_purchase_store,num_purchase_discount,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,revenue
0,5524,2020-12,1965,75579400.0,미혼,0,58,825500,114400,709800,...,8,4,3,0,0,0,0,0,1,11
1,2174,2022-06,1962,60247200.0,미혼,2,38,14300,1300,7800,...,1,2,2,0,0,0,0,0,0,11
2,4141,2021-11,1973,93096900.0,배우자 있음,0,26,553800,63700,165100,...,8,10,1,0,0,0,0,0,0,11
3,6182,2022-05,1992,34639800.0,배우자 있음,1,26,14300,5200,26000,...,2,4,2,0,0,0,0,0,0,11
4,5324,2022-04,1989,75780900.0,배우자 있음,1,94,224900,55900,153400,...,5,6,5,0,0,0,0,0,0,11


## Data Preparation

In [97]:
# Check data types for each column
df.dtypes

birth_year                 int64
annual_income            float64
marital_status            object
children                   int64
recency                    int64
amount_alcohol             int64
amount_fruit               int64
amount_meat                int64
amount_fish                int64
amount_snack               int64
amount_general             int64
num_purchase_web           int64
num_purchase_store         int64
num_purchase_discount      int64
promotion_1                int64
promotion_2                int64
promotion_3                int64
promotion_4                int64
promotion_5                int64
promotion_6                int64
revenue                    int64
dtype: object

Before diving into looking at the data, there are a few things I took care of:\
1) Convert values for 'martial_status' column from Korean to English\
2) Drop 'ID' and 'signup_ym' columns since they do not affect the analysis

In [69]:
# Find out values in marital_status column
df['marital_status'].unique()

array(['미혼', '배우자 있음', '이혼', '사별'], dtype=object)

In [70]:
# Create a dictionary to replace the Korean values to English
marital_status = {'미혼': 'single',
                 '배우자 있음': 'married',
                 '이혼': 'divorced',
                 '사별': 'widowed'}

# use .replace() to replace the values
df['marital_status'].replace(marital_status, inplace=True)

In [71]:
# Check whether the values are replaced correctly
df['marital_status'].unique()

array(['single', 'married', 'divorced', 'widowed'], dtype=object)

In [72]:
# Drop 'ID' and 'signup_ym'
df.drop(['ID', 'signup_ym'], axis=1, inplace=True)

Now that the dataset is correctly set, I started looking at the dataset.

In [73]:
# Check how many records are in the dataset
len(df)

2240

In [74]:
# Check for any missing values
df.isnull().sum()

birth_year                0
annual_income            24
marital_status            0
children                  0
recency                   0
amount_alcohol            0
amount_fruit              0
amount_meat               0
amount_fish               0
amount_snack              0
amount_general            0
num_purchase_web          0
num_purchase_store        0
num_purchase_discount     0
promotion_1               0
promotion_2               0
promotion_3               0
promotion_4               0
promotion_5               0
promotion_6               0
revenue                   0
dtype: int64

I found out that there are 24 missing values in the "annual income" column.\
I now had to decide whether to drop the rows or to impute the missing data.

In [75]:
df[df['annual_income'].isnull()]

,birth_year,annual_income,marital_status,children,recency,amount_alcohol,amount_fruit,amount_meat,amount_fish,amount_snack,...,num_purchase_web,num_purchase_store,num_purchase_discount,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,revenue
10,1991,NaN,married,1,11,6500,6500,7800,0,2600,...,1,2,1,0,0,0,0,0,0,11
27,1994,NaN,single,1,19,6500,1300,3900,3900,341900,...,27,0,0,0,0,0,0,0,0,11
43,1967,NaN,single,0,80,105300,14300,65000,3900,2600,...,1,4,1,0,0,0,0,0,0,11
48,1959,NaN,single,3,96,62400,6500,62400,7800,13000,...,2,4,3,0,0,0,0,0,0,11
58,1990,NaN,single,1,57,14300,3900,28600,2600,2600,...,2,3,2,0,0,0,0,0,0,11
71,1981,NaN,married,1,25,32500,3900,55900,22100,5200,...,3,3,3,0,0,0,0,0,0,11
90,1965,NaN,married,3,4,299000,54600,249600,63700,48100,...,7,8,12,0,0,0,0,0,0,11
91,1965,NaN,single,2,45,9100,0,10400,2600,0,...,1,2,1,0,0,0,0,0,0,11
92,1981,NaN,married,0,87,578500,48100,466700,127400,36400,...,2,8,1,0,0,0,0,0,0,11
128,1969,NaN,married,1,23,457600,0,35100,13000,0,...,6,7,3,0,0,0,0,0,0,11


I decided to impute the missing annual income data using the KNN imputer because the missing values are small in porpotion, and I believe that there are sufficient data to impute annual income using features like marital status, number of children, birth year, etc. \
To prepare the dataset for KNNImputer, I did the following:\
1) One-hot encode martial status
2) Properly scale the data

In [76]:
# One-hot encoding 'marital_status column'
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

,birth_year,annual_income,children,recency,amount_alcohol,amount_fruit,amount_meat,amount_fish,amount_snack,amount_general,...,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,revenue,marital_status_married,marital_status_single,marital_status_widowed
0,1965,75579400.0,0,58,825500,114400,709800,223600,114400,114400,...,0,0,0,0,0,1,11,0,1,0
1,1962,60247200.0,2,38,14300,1300,7800,2600,1300,7800,...,0,0,0,0,0,0,11,0,1,0
2,1973,93096900.0,0,26,553800,63700,165100,144300,27300,54600,...,0,0,0,0,0,0,11,1,0,0
3,1992,34639800.0,1,26,14300,5200,26000,13000,3900,6500,...,0,0,0,0,0,0,11,1,0,0
4,1989,75780900.0,1,94,224900,55900,153400,59800,35100,19500,...,0,0,0,0,0,0,11,1,0,0


In [92]:
# Scale the data
scaler = MinMaxScaler()
df_norm = scaler.fit_transform(df_encoded)

In [93]:
# KNN Imputer
imputer = KNNImputer()
imputed = imputer.fit_transform(df_norm)

In [94]:
df_processed = pd.DataFrame(imputed, columns = df_encoded.columns)
df_processed.head()

,birth_year,annual_income,children,recency,amount_alcohol,amount_fruit,amount_meat,amount_fish,amount_snack,amount_general,...,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,revenue,marital_status_married,marital_status_single,marital_status_widowed
0,0.621359,0.084832,0.000000,0.585859,0.425318,0.442211,0.316522,0.664093,0.334601,0.243094,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.592233,0.067095,0.666667,0.383838,0.007368,0.005025,0.003478,0.007722,0.003802,0.016575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.699029,0.105097,0.000000,0.262626,0.285332,0.246231,0.073623,0.428571,0.079848,0.116022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.883495,0.037471,0.333333,0.262626,0.007368,0.020101,0.011594,0.038610,0.011407,0.013812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.854369,0.085065,0.333333,0.949495,0.115874,0.216080,0.068406,0.177606,0.102662,0.041436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [95]:
# Inverse Scale the Dataset
df_imputed = scaler.inverse_transform(df_processed)
df_imputed = pd.DataFrame(df_imputed, columns = df_processed.columns)
df_imputed.head()

,birth_year,annual_income,children,recency,amount_alcohol,amount_fruit,amount_meat,amount_fish,amount_snack,amount_general,...,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,revenue,marital_status_married,marital_status_single,marital_status_widowed
0,1965.0,75579400.0,0.0,58.0,825500.0,114400.0,709800.0,223600.0,114400.0,114400.0,...,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0,1.0,0.0
1,1962.0,60247200.0,2.0,38.0,14300.0,1300.0,7800.0,2600.0,1300.0,7800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,1.0,0.0
2,1973.0,93096900.0,0.0,26.0,553800.0,63700.0,165100.0,144300.0,27300.0,54600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0
3,1992.0,34639800.0,1.0,26.0,14300.0,5200.0,26000.0,13000.0,3900.0,6500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0
4,1989.0,75780900.0,1.0,94.0,224900.0,55900.0,153400.0,59800.0,35100.0,19500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0


In [99]:
# Check if missing values are correctly imputed
df_imputed.isnull().sum()

birth_year                0
annual_income             0
children                  0
recency                   0
amount_alcohol            0
amount_fruit              0
amount_meat               0
amount_fish               0
amount_snack              0
amount_general            0
num_purchase_web          0
num_purchase_store        0
num_purchase_discount     0
promotion_1               0
promotion_2               0
promotion_3               0
promotion_4               0
promotion_5               0
promotion_6               0
revenue                   0
marital_status_married    0
marital_status_single     0
marital_status_widowed    0
dtype: int64

In [100]:
# Export dataset
df_imputed.to_csv('customer_data_processed.csv', index=False)